In [1]:
# load needed modules
import numba 
from numba import cuda
import numpy as np


In [ ]:
# ================================= median ================================== #

'''
 @param:
     - input_data:  np.array, 3D data to filter
     - output_data: np.array, 3D data after filtering,same size of input (because cuda kernel function can't return value)
     - fkt:         function, numba_median, numba_avgerage, ... (what else?) ==> Gaussian?
     - stencil:     tuple of size 3, the dimension of 3D stencil 
     
 @function: 
     stencil filter on GPU, reference: https://en.wikipedia.org/wiki/Stencil_code
'''

@cuda.jit
def gpu_smooth_kernel_naive(input_data, output_data, stencil_z, stencil_y, stencil_x, Nz, Ny, Nx):
    
    # ==== stencil size ==== #
    #dx = stencil_[0] dosen't work, compile issue
    #dy = stencil_[1]
    #dz = stencil_[2]
    dx = stencil_x
    dy = stencil_y
    dz = stencil_z
    # ====================== #
    
    row_init = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    col_init = cuda.threadIdx.y + cuda.blockIdx.y * cuda.blockDim.y
    depth_init = cuda.threadIdx.z + cuda.blockIdx.z * cuda.blockDim.z
    
    # grid stride:
    for depth_global in range(depth_init, Nz, cuda.blockDim.z * cuda.gridDim.z):
        z_min = max(depth_global - stencil_z, 0)
        z_max = min(depth_global + stencil_z + 1, Nz)
        for col_global in range(col_init, Ny, cuda.blockDim.y * cuda.gridDim.y):
            y_min = max(col_global - stencil_y, 0)
            y_max = min(col_global + stencil_y + 1, Ny)
            for row_global in range(row_init, Nx, cuda.blockDim.x * cuda.gridDim.x):
                
                x_min = max(row_global - stencil_x, 0)
                x_max = min(row_global + stencil_x + 1, Nx)
                
                # define local array in cuda: 
                # https://stackoverflow.com/questions/48642481/what-is-the-correct-usage-of-cuda-local-array-in-numba
                sort_array = cuda.local.array(10 * 8 * 10, numba.float32) 
                
                # load in local memory to sort
                n = 0
                for i in range(z_min,z_max):
                    for j in range(y_min,y_max):
                        for k in range(x_min,x_max):
                            sort_array[n] = input_data[i,j,k]
                            n += 1
                
                # selection sort:
                # https://www.cnblogs.com/BobHuang/p/11263183.html <== chinese website
                for i in range(n - 1):
                    min_index = i;                  
                    for j in range(i+1, n):
                        if (sort_array[j] < sort_array[min_index]):
                            min_index = j; 
                
                    #swap(sort_array[i], sort_array[minIndex]);
                    tmp = sort_array[i]
                    sort_array[i] = sort_array[min_index]
                    sort_array[min_index] = tmp
                    
                half = int(n / 2)
                if (n % 2) == 1:
                    median = sort_array[half]
                else:
                    median = (sort_array[half-1] + sort_array[half]) / 2.0
             
                output_data[depth_global, col_global, row_global] = median
                

        
def lauch_kernel(input_data, output_data, stencil_):
    # TODO: set blocksize, gridsize and lauch kernel
    # define threads and blocks
    threads = (16, 8, 4)
    blocks = (8, 8, 8)
    print("==================")
    print("threads: " + str(threads))
    print("blocks: " + str(blocks))
    print("==================")
    
    Nz, Ny, Nx = np.shape(input_data) # why here z, y, x ?
    stencil_z, stencil_y, stencil_x = stencil_[0], stencil_[1], stencil_[2]
  
    # call CUDA kernel
    gpu_smooth_kernel_naive[threads, blocks](input_data, output_data,stencil_z, stencil_y, stencil_x, Nz, Ny, Nx)
    
